# Program Description: XAS Simulation (Module 1)

## 1. Module Overview
This module takes a structure file in `.xyz` format as input, containing `n` 3D structures (where `n` represents the number of samples). It generates an input file for each structure using the `pymatgen` library and runs the `feff.run()` function to calculate the absorption spectrum.

## 2. Calculated Spectra and Operations
The module extracts the following data from the FEFF results:
- **χ (Chi)**
- **μ (Xmu)**
- **FT** (Fourier Transform)
- **BFT** (Back Fourier Transform)
- **WT** (Wavelet Transform)
- **WT_mag** (Magnitude of Wavelet Transform)

Each of these operations is saved in a separate output file.

## 3. FEFF Calculation Parameters
Note: The FEFF calculation parameters are not read from an external input file; all settings are defined directly within the code. Users can modify the FEFF calculation parameters by adjusting the relevant settings in the program.

## 4. Output
The calculated spectra are saved in a file named **"datasets"** in the current directory. The structure and spectra data are organized accordingly for each sample.

## 5. Progress Monitoring
This program includes a progress bar to provide real-time feedback during the calculation process. The progress bar displays three key pieces of information:
- **Time elapsed**: The time that has passed since the start of the calculation.
- **Estimated remaining time**: The projected time left for the current calculation to complete.
- **Average calculation time per sample**: The average time taken for each sample's calculation.

## 6. Parameters
- **`num_processes`**: Sets the number of processes used for parallel computation.
- **`n`**: Defines the total number of samples to be processed.
m.

contacts: zhaohf@ihep.ac.cn

# Import required libraries

In [1]:
import sys
import pymatgen.io.ase as aseio
from numpy import mean
import pkg_resources
#from larch.io import read_ascii
from larch import Group
import numpy as np
import matplotlib.pyplot as plt
from numpy import array
from os import mkdir, getcwd, chdir, remove
from ase.io import read
from shutil import copyfile, rmtree
from os.path import exists, join, abspath
from pymatgen.io.xyz import XYZ
from pymatgen.core import Structure
from pymatgen.io.feff.sets import FEFFDictSet
from larch.xafs import feffrunner
from pymatgen.core import Lattice, Structure, Molecule
from larch.xafs import autobk, xftf, xftr, cauchy_wavelet,pre_edge
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import multiprocessing
from tqdm import tqdm
from pymatgen.io.ase import AseAtomsAdaptor
from concurrent.futures import ThreadPoolExecutor
import os
from shutil import copyfile, rmtree,move
import traceback
import glob
import re
from os.path import basename
import tempfile
from multiprocessing import Pool, Lock
from ase.build import bulk
from concurrent.futures import TimeoutError


## The version of each package

In [2]:
def get_python_version():
    return sys.version

def get_package_version(package_name):
    try:
        module = __import__(package_name)
        version = getattr(module, '__version__', None)
        if version:
            return version
        else:
            return pkg_resources.get_distribution(package_name).version
    except (ImportError, AttributeError, pkg_resources.DistributionNotFound):
        return "Version info not found"

packages = [
    'numpy',
    'matplotlib',
    'pandas',
    'sklearn', 
    'pymatgen',
    'ase',
    'larch',  
    'concurrent.futures',  
    'shutil',  
    'os', 
    'glob',  
    'tempfile', 
    're', 
]
for package in packages:
    print(f"{package}: {get_package_version(package)}")
print(f"Python: {get_python_version()}")

numpy: 1.23.5
matplotlib: 3.7.5
pandas: 2.0.3
sklearn: 1.3.2
pymatgen: 2023.8.10
ase: 3.22.0
larch: 0.9.76
concurrent.futures: Version info not found
shutil: Version info not found
os: Version info not found
glob: Version info not found
tempfile: Version info not found
re: 2.2.1
Python: 3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:46:39) 
[GCC 10.4.0]


#  Parameter settings

## Input File:
- **`read_single_file`**: 
  - If set to `True`, all samples are read from a single trajectory file.
  - If set to `False`, the program reads each sample from individual files, as specified in the **`file_paths`** list.

## Output File:
- The program will save the results in the specified output directory. Each result will be saved in a separate file according to the processing steps defined in the program.

## Process Parameters:
- **`num_processes`**: 
  - Defines the number of processes to be used for parallel computation.
  - This allows for multi-threaded processing, improving computation time for larger datasets.
  
- **`n`**: 
  - Specifies the total number of samples to be processed by the program.
  - This is typically the number of data points or structures to be analyzed in the simulation.
culated.

In [3]:
# Flag to specify whether all samples should be read from a single file
read_single_file = True

# Set the file format for reading data. Options include 'vasp-xdatcar' or 'lammps-dump-text'
format = "lammps-dump-text"

# Path to the input data file
data_input = "Cu-20220624.xyz"

# Get and display the current working directory
current_directory = os.getcwd()
print(f"Current working directory: {current_directory}")

# Define directory paths for storing calculated data
data_dir = "0120-datasets"  # Base directory for datasets
chi_dir = os.path.join(data_dir, "chi")  # Directory for chi data
xmu_dir = os.path.join(data_dir, "xmu")  # Directory for xmu data
wt_dir = os.path.join(data_dir, "wt")  # Directory for wt data
wt_pic_dir = os.path.join(data_dir, "wt_pic")  # Directory for wt pictures
norm_dir = os.path.join(data_dir, "norm")  # Directory for normalized data
inp_dir = os.path.join(data_dir, "inp")  # Directory for input data
temp_dir = os.path.join(data_dir, "temp")  # Directory for temporary files

# Set the number of samples to process
n = 30  # Total number of samples to calculate

# Define the X-ray absorption edge (e.g., 'L3')
edge = 'L3'

# Set the number of parallel processes for computation
num_processes = 5


Current working directory: /media/dell-hd/data1/hanxue/xasdaml0120


In [4]:
def extract_number(filename):
    """Extract the largest number from the filename using regular expressions."""
    # Find all numeric values in the filename
    match = re.findall(r'\d+', os.path.basename(filename))
    # Convert all found numeric parts to integers and return the largest number
    numbers = [int(num) for num in match]
    return max(numbers) if numbers else float('inf')  # Return the largest number or infinity if no numbers are found

# Read the trajectory file for the vasp-xdatcar format
if read_single_file:
    # Read the entire file containing all samples
    traj = read(data_input, format=format, index=':')
    total_samples = len(traj)
else:
    file_paths = sorted(glob.glob(join(data_input, '*.xyz')), key=extract_number)
    print("Sorted file paths:")
    for file_path in file_paths:
        print(file_path)
    # Read each trajectory file
    trajectories = [read(file_path, format=format) for file_path in file_paths]
    total_samples = len(trajectories)

def if_dirs(dirs):
    """Check if directories exist, and create them if they do not."""
    for d in dirs:
        if not os.path.exists(d):
            os.mkdir(d)
            print(f"Directory '{d}' created successfully.")
        else:
            print(f"Directory '{d}' already exists, skipping creation.")
    return True

# Verify the existence of required directories and create them if necessary
if_dirs([data_dir, chi_dir, xmu_dir, wt_dir, wt_pic_dir, norm_dir, inp_dir, temp_dir])

# Determine the number of available CPUs
cpu_count = multiprocessing.cpu_count()

# Print the number of CPUs and the total number of samples to be processed
num_samples_to_process = min(n, total_samples)
print(f"Number of available CPUs: {cpu_count}")
print(f"Total number of samples to be processed: {total_samples}")

Directory '0120-datasets' created successfully.
Directory '0120-datasets/chi' created successfully.
Directory '0120-datasets/xmu' created successfully.
Directory '0120-datasets/wt' created successfully.
Directory '0120-datasets/wt_pic' created successfully.
Directory '0120-datasets/norm' created successfully.
Directory '0120-datasets/inp' created successfully.
Directory '0120-datasets/temp' created successfully.
Number of available CPUs: 64
Total number of samples to be processed: 5001


## FEFF Parameter Setup

In [5]:
def get_inp(absorbing_atom, structure, radius, save_dir):
    """
    Generate the FEFF input file based on the provided parameters.

    Parameters:
        absorbing_atom (str): The element symbol of the absorbing atom.
        structure (Structure): The atomic structure for the FEFF calculation.
        radius (float): The radius around the absorbing atom for the calculation.
        save_dir (str): The directory where the FEFF input file will be saved.

    Returns:
        str: The path to the generated FEFF input file.
    """
    # Configuration dictionary for the FEFF calculation
    config = {
        'CONTROL': '1 1 1 1 1 1',  # Control flags for various calculations
        'EDGE': edge,  # The X-ray absorption edge (e.g., 'L3')
        'EXAFS': 20.0,  # EXAFS calculation distance range
        'PRINT': '0 0 0 0 0 0',  # Control print options
        'RPATH': 7.0,  # Maximum path length for multiple scattering
        'S02': 1.0,  # Amplitude reduction factor
        'SCF': '6.8 0 30 0.1000 1',  # SCF calculation parameters
        'EXCHANGE': '0 0 0.6',  # Exchange-correlation options
        'NLEGS': '8',  # Maximum number of legs for multiple scattering paths
        'DEBYE': '300 160',  # Debye-Waller factor parameters
        'CFAVERAGE': '1 1 0',  # Cluster configuration averaging options
        'CRITERIA': '0.0 1.50'  # Path selection criteria
    }
    
    # Generate the FEFF input using the provided configuration and structure
    feff_inp = FEFFDictSet(
        absorbing_atom=absorbing_atom,
        structure=structure,
        radius=radius,
        config_dict=config,
        edge=edge,
        nkpts=100,  # Number of k-points for sampling
        user_tag_settings={"_del": ["COREHOLE"]}  # Custom tag settings
    )
    
    # Write the input file to the specified directory
    feff_inp.write_input(save_dir)
    
    # Return the path to the generated FEFF input file
    return join(save_dir, "feff.inp")

# Display the number of samples to be processed and the number of parallel processes
print(f"Number of samples to be processed: {num_samples_to_process}")
print(f"Number of processes to use for parallel computation: {num_processes}")


Number of samples to be processed: 30
Number of processes to use for parallel computation: 5


# Function settings

In [6]:
def Analysis_xmu(filepath):
    """Parse the xmu data file and extract key variables: omega, energy (e), wave vector (k), 
    absorption coefficient (mu), reference absorption (mu0), and the chi function.
    
    The parsed data is then returned as a Group object for further analysis.
    
    Args:
        filepath (str): Path to the xmu data file.

    Returns:
        Group: A Group object containing omega, energy, k, mu, mu0, and chi as arrays.
    """
    file = open(filepath, "r").read().splitlines()
    omega, e, k, mu, mu0, chi = [], [], [], [], [], []
    for line in file:
        if line[0] == "#":  # Skip comment lines
            continue
        temp = line.split()
        omega.append(float(temp[0]))
        e.append(float(temp[1]))
        k.append(float(temp[2]))
        mu.append(float(temp[3]))
        mu0.append(float(temp[4]))
        chi.append(float(temp[5]))
    
    data = Group(omega=array(omega), 
                 e=array(e), 
                 k=array(k), 
                 mu=array(mu), 
                 mu0=array(mu0), 
                 chi=array(chi))
    return data


def save_chi(k, chi, save_path):
    """Save the chi function data to a CSV file.

    Args:
        k (array): Wave vector (k) values.
        chi (array): Chi function values.
        save_path (str): Path where the CSV file will be saved.

    Returns:
        bool: True if the data is saved successfully, otherwise False.
    """
    data = pd.DataFrame()
    data["k"] = k
    data["chi"] = chi
    data.to_csv(save_path, index=False)     
    return True


def save_xmu(energy, mu, save_path):
    """Save the xmu data to a CSV file.

    Args:
        energy (array): Energy values.
        mu (array): Absorption coefficient values.
        save_path (str): Path where the CSV file will be saved.

    Returns:
        bool: True if the data is saved successfully, otherwise False.
    """
    data = pd.DataFrame()
    data["energy"] = energy
    data["mu"] = mu
    data.to_csv(save_path, index=False)  
    return True


def save_norm(energy, norm, save_path):
    """Save the normalized data to a CSV file.

    Args:
        energy (array): Energy values.
        norm (array): Normalized values.
        save_path (str): Path where the CSV file will be saved.

    Returns:
        bool: True if the data is saved successfully, otherwise False.
    """
    data = pd.DataFrame()
    data["energy"] = energy
    data["norm"] = norm
    data.to_csv(save_path, index=False)  
    return True


def save_wt(k, wcauchy_r, wcauchy_re, wcauchy_im, wcauchy_mag, save_path):
    """Save the results of the wavelet transform as a CSV file.

    This function processes the wavelet transform results (real and imaginary parts, magnitude) 
    and saves them in a CSV format. It also filters the k and r values for better representation.

    Args:
        k (array): Wave vector (k) values.
        wcauchy_r (array): Radial distances (r) for the wavelet transform.
        wcauchy_re (array): Real part of the wavelet transform.
        wcauchy_im (array): Imaginary part of the wavelet transform.
        wcauchy_mag (array): Magnitude of the wavelet transform.
        save_path (str): Path where the CSV file will be saved.

    Returns:
        bool: True if the data is saved successfully, otherwise False.
    """
    data = []
    r = wcauchy_r
    for i in range(len(r)):
        for j in range(len(k)):
            data.append([k[j],
                         r[i],
                         wcauchy_re[i][j],
                         wcauchy_im[i][j],
                         wcauchy_mag[i][j]])
    
    columns = ["k", "r", "re", "im", "mag"]
    data = pd.DataFrame(data, columns=columns)
    
    # Filtering k and r values for better representation
    k_list, r_list = sorted(list(set(data.k.values))), sorted(list(set(data.r.values)))
    if 0 in k_list: k_list.remove(0.0)
    k_list, r_list = k_list[::2], r_list[::2]

    data = data[data['k'].isin(k_list)]
    data = data[data['r'].isin(r_list)]
    
    # Save the filtered data
    data.to_csv(save_path, index=False)  
    return True


def plot_wt_mag(k, wcauchy_r, wcauchy_mag, save_path):
    """Plot and save a 2D contour plot of the wavelet transform magnitude.

    This function generates a contour plot of the magnitude of the wavelet transform
    and saves it as an image file.

    Args:
        k (array): Wave vector (k) values.
        wcauchy_r (array): Radial distances (r) for the wavelet transform.
        wcauchy_mag (array): Magnitude of the wavelet transform.
        save_path (str): Path where the plot image will be saved.

    Returns:
        bool: True if the plot is saved successfully, otherwise False.
    """
    r = wcauchy_r
    # Create the contour plot
    plt.figure(figsize=(8, 8.5), dpi=360)
    x, y = np.meshgrid(k, r)
    z = wcauchy_mag
    plt.contourf(x, y, z, cmap=plt.cm.bwr, levels=40)
    
    frame = plt.gca()
    frame.axes.get_yaxis().set_visible(False)
    frame.axes.get_xaxis().set_visible(False)
    
    # Save the plot
    plt.savefig(save_path)
    plt.close()
    return True


def run_feff(args):
    """Run FEFF calculation using the specified input parameters.

    This function generates a FEFF input file, runs the FEFF simulation, 
    and restores the working directory after execution.

    Args:
        args (tuple): A tuple containing the parameters: 
                      absorbing_atom, structure, radius, temp_dir, inp_dir.

    Returns:
        None
    """
    absorbing_atom, structure, radius, temp_dir, inp_dir = args
    temp_cwd = os.getcwd()  # Store the current working directory
    inp = get_inp(absorbing_atom=absorbing_atom, structure=structure, radius=radius, save_dir=temp_dir)
    
    # Run FEFF simulation
    feff = feffrunner(feffinp=abspath(inp), folder=temp_dir)
    feff.run()
    
    # Restore the working directory
    os.chdir(temp_cwd)
    return


# Main program

In [7]:
# Lock for synchronizing access to shared resources during multiprocessing
lock = multiprocessing.Lock()

def process_sample(j):
    """
    Process a single sample in parallel. This includes running FEFF, analyzing the data,
    and saving the results.

    Args:
        j (int): Index of the sample to process.
    """
    try:
        # Log the current sample being processed
        print(f"Processing sample {j}")
        
        # Create a temporary directory for storing intermediate results of the current sample
        sample_temp_dir = os.path.join(temp_dir, f"temp_{j}")
        os.makedirs(sample_temp_dir, exist_ok=True)

        # Read the sample data, either from a single file or from individual files
        if read_single_file:
            # Ensure the index is within the bounds of the trajectory data
            if j >= len(traj):
                raise IndexError(f"Index {j} is out of bounds for traj with length {len(traj)}")
            frame = traj[j]
        else:
            # Read the trajectory file corresponding to the sample index
            sample_file = file_paths[j]
            frame = read(sample_file, format=format)

        # Set the unit cell size for the atomic structure (e.g., 40 Å³ in each dimension)
        frame.set_cell((40, 40, 40))
        structure = AseAtomsAdaptor.get_structure(frame)

        print(f"Running FEFF for sample {j}")
        
        # Define the absorbing atom (e.g., atom index 0)
        absorbing_atom = 0
        if absorbing_atom >= len(structure):
            raise IndexError(f"Absorbing atom index {absorbing_atom} is out of bounds for structure with {len(structure)} atoms")

        # Generate the FEFF input file and run the FEFF simulation
        run_feff((absorbing_atom, structure, 20, sample_temp_dir, inp_dir))

        # Check for the xmu output file from the FEFF run
        temp_cwd = os.getcwd()  # Store the current working directory
        xmu_path = os.path.join(sample_temp_dir, "xmu.dat")
        if not os.path.exists(xmu_path):
            raise FileNotFoundError(f"xmu.dat not found for sample {j} in {sample_temp_dir}")

        # Analyze the xmu data and extract the relevant XAS information
        xmu = Analysis_xmu(xmu_path)
        xas = Group(energy=xmu.omega, mu=xmu.mu)

        print(f"Processing FEFF results for sample {j}")
        
        # Perform background subtraction and XAS analysis
        autobk(xas, rbkg=0.85, kmin=0.5, kmax=19.8, kstep=1.5, kweight=2)
        
        # Prepare the data for wavelet transformation
        xaswt = Group(k=xmu.k, chi=xmu.chi)
        cauchy_wavelet(xaswt, kweight=2, rmax_out=8)

        # Save the results using a lock to ensure synchronized access to shared directories
        with lock:
            # Save FEFF input file, chi, xmu, norm, wavelet transform, and plots
            copyfile(os.path.join(sample_temp_dir, "feff.inp"), os.path.join(inp_dir, f"{j}.inp"))
            save_chi(xmu.k, xmu.chi, os.path.join(chi_dir, f"{j}.csv"))
            save_xmu(xas.energy, xas.mu, os.path.join(xmu_dir, f"{j}.csv"))
            save_norm(xas.energy, xas.flat, os.path.join(norm_dir, f"{j}.csv"))
            save_wt(xaswt.k, xaswt.wcauchy_r, xaswt.wcauchy_re, xaswt.wcauchy_im, xaswt.wcauchy_mag, os.path.join(wt_dir, f"{j}.csv"))
            plot_wt_mag(xaswt.k, xaswt.wcauchy_r, xaswt.wcauchy_mag, os.path.join(wt_pic_dir, f"{j}.png"))

        # Restore the working directory after processing
        os.chdir(temp_cwd)

    except Exception as e:
        # Handle exceptions and log errors for the current sample
        print(f"Error processing sample {j}: {e}")
        traceback.print_exc()
    finally:
        # Clean up: Remove the temporary directory for the current sample
        rmtree(sample_temp_dir)
        sys.stdout.flush()

# Validate the number of samples in the input files
if read_single_file:
    if len(traj) != total_samples:
        print(f"Warning: traj length {len(traj)} does not match total_samples {total_samples}")
else:
    if len(file_paths) != total_samples:
        print(f"Warning: file_paths length {len(file_paths)} does not match total_samples {total_samples}")

# Create a process pool and process the samples in parallel
with multiprocessing.Pool(processes=num_processes) as pool:
    # Display a progress bar to monitor the sample processing progress
    with tqdm(total=num_samples_to_process, desc='Computing Samples') as pbar:
        # Use imap_unordered for parallel processing
        for _ in pool.imap_unordered(process_sample, range(num_samples_to_process)):
            pbar.update(1)

# Close the process pool once all tasks are completed
pool.close()
pool.join()


Computing Samples:   0%|                                 | 0/30 [00:00<?, ?it/s]

Processing sample 0Processing sample 2Processing sample 1Processing sample 3Processing sample 4




Running FEFF for sample 0
Running FEFF for sample 2
Running FEFF for sample 3
Running FEFF for sample 1Running FEFF for sample 4

 : ======== running Feff module feff8l_rdinp ========
 : ======== running Feff module feff8l_rdinp ========
 : ======== running Feff module feff8l_rdinp ========
 : ======== running Feff module feff8l_rdinp ========
 : Feff8L (EXAFS)      release  0.1
 : ======== running Feff module feff8l_rdinp ========
 : comment: None given
 : Feff8L (EXAFS)      release  0.1
 : Feff8L (EXAFS)      release  0.1
 : Source:
 : comment: None given
 : comment: None given
 : Feff8L (EXAFS)      release  0.1
 : Structure Summary:  Cu108
 : Source:
 : Feff8L (EXAFS)      release  0.1
 : Source:
 : comment: None given
 : Reduced formula:  Cu
 : Structure Summary:  Cu108
 : comment: None given
 : Structure Summary:  Cu108
 : Source:
 : space group: (P1), space number:  (1)
 : Reduce

Computing Samples:   3%|▊                        | 1/30 [00:54<26:18, 54.43s/it]

Processing sample 5
Running FEFF for sample 5
 : ======== running Feff module feff8l_rdinp ========
 : Feff8L (EXAFS)      release  0.1
 : comment: None given
 : Source:
 : Structure Summary:  Cu108
 : Reduced formula:  Cu
 : space group: (P1), space number:  (1)
 : abc: 40.000000  40.000000  40.000000
 : angles: 90.000000  90.000000  90.000000
 : sites: 108
 : ======== running Feff module feff8l_pot ========
 : point #  40  energy =  -9.015
 : Calculating potentials ...
 : free atom potential and density for atom type    0
 : free atom potential and density for atom type    1
 : initial state energy
 : overlapped potential and density for unique potential    0
 : overlapped potential and density for unique potential    1
 : muffin tin radii and interstitial parameters
 : : ipot, Norman radius, Muffin tin radius, Overlap
 : 0  1.80581E+00  1.62972E+00  1.15000E+00
 : 1  1.44679E+00  1.37236E+00  1.14488E+00
 : Core-valence separation
 : : mu_old=    -3.656
 : SCF ITERATION NUMBER  1  O

Computing Samples:   7%|█▋                       | 2/30 [02:09<30:56, 66.31s/it]

...

Computing Samples:  93%|██████████████████████▍ | 28/30 [12:01<00:41, 20.86s/it]

 : 0   FMS matrix (LUD) at point   1, number of state kets = 342


Computing Samples:  97%|███████████████████████▏| 29/30 [12:02<00:14, 14.88s/it]

 : point #  20  energy = -28.979
 : point #  40  energy =  -8.940
 : Electronic configuration
 : iph    il      N_el
 : 0     0    1.171
 : 0     1    1.061
 : 0     2   10.190
 : 0     3    0.000
 : 0     4    0.000
 : 1     0    0.747
 : 1     1    0.679
 : 1     2    9.574
 : 1     3    0.000
 : 1     4    0.000
 : mu_new=    -7.938
 : Charge transfer:  iph  charge(iph)
 : 0   -0.354
 : 1    0.000
 : SCF ITERATION NUMBER  7  OUT OF 30
 : Calculating energy and space dependent l-DOS.
 : It takes time ...
 : point #   1  energy = -40.000
 : Doing FMS for a cluster of  26 atoms around iph =  0
 : 0   FMS matrix (LUD) at point   1, number of state kets = 234
 : Doing FMS for a cluster of  38 atoms around iph =  1
 : 0   FMS matrix (LUD) at point   1, number of state kets = 342
 : point #  20  energy = -28.979
 : point #  40  energy =  -8.940
 : Electronic configuration
 : iph    il      N_el
 : 0     0    1.159
 : 0     1    1.049
 : 0     2   10.182
 : 0     3    0.000
 : 0     4    0.

Computing Samples: 100%|████████████████████████| 30/30 [12:58<00:00, 25.96s/it]
